In [9]:
import tweepy
from tweepy import OAuthHandler
import datetime
import dateutil.parser as parser
import numpy as np
import pandas as pd
from my_profile import my_profile

In [13]:
def my_profile():
    my_profile = api.me()

    user = my_profile.name
    loc = my_profile.location
    id_me = my_profile.id
    created = my_profile.created_at.date()
    profile_img = my_profile.default_profile_image
    favs = my_profile.favourites_count
    followers = my_profile.followers_count
    friends = my_profile.friends_count
    geo = my_profile.geo_enabled
    profile_img = my_profile.profile_image_url_https
    background_img = my_profile.profile_background_image_url_https
        
    me = {'Id': id_me, 'Name': user, 'Location': loc, 'Created': created, 'Profile image': profile_img, 'Favs count': favs,
          'Followers': followers, 'Following': friends, 'Geolocation': geo, 'Profile image': profile_img,
          'Background image': background_img}
    return me


def hashtags_fn(tweet_i):
    if len(tweet_i.entities.get('hashtags')) != 0:
        hashtags = []
        for hstg in tweet_i.entities.get('hashtags'):
            hashtags.append(hstg.get('text'))
    else:
        hashtags = None
    return hashtags

def user_mentions_fn(tweet_i):
    if len(tweet_i.entities.get('user_mentions')) != 0:
        mentions = []
        for mention in tweet_i.entities.get('user_mentions'):
            mentions.append(mention.get('screen_name'))
    else:
        mentions = None
    return mentions

def urls_fn(tweet_i):
    if len(tweet_i.entities.get('urls')) != 0:
        urls = []
        for url_info in tweet_i.entities.get('urls'):
            urls.append(url_info.get('expanded_url'))
    else:
        urls = None
    return urls

def urls_user(profile):
    if profile.entities.get('url').get('urls') != 0:
        urls = []
        for url_info in profile.entities.get('url').get('urls'):
            urls.append(url_info.get('expanded_url'))
    else:
        urls = None
    return urls

def media_fn(tweet_i):
    if tweet_i.entities.get('media') != None:
        media = []
        for media_info in tweet_i.entities.get('media'):
            media.append(media_info.get('media_url_https'))
    else:
        media = None
    return media

def get_info_tweet(tweet_i):
    tweet_dict = {'Date': tweet_i.created_at,
                  'ID Tweet': tweet_i.id,
                  'User': tweet_i.author._json.get('name'),
                  'Tweet': tweet_i.text,
                  'Language': tweet_i.lang,
                  'Retweet': tweet_i.retweet_count,
                  'Fav': tweet_i.favorite_count,
                  'Reply to id': tweet_i.in_reply_to_user_id,
                  'Reply to name': tweet_i.in_reply_to_screen_name,
                  'Coordinates': tweet_i.coordinates, 
                  'Hastags': hashtags_fn(tweet_i),
                  'URLs': urls_fn(tweet_i),
                  'User mentions': user_mentions_fn(tweet_i),
                  'Images/videos': media_fn(tweet_i)}
    return tweet_dict

def last_n_tweets(n):
    tweets = []
    for tweet in tweepy.Cursor(api.home_timeline).items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def last_n_tweets_user(user, n):
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, id = user).items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)

def last_n_tweets_word(word, n):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q = f"\"{word}\"", lang = "es").items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def last_n_tweets_city(city, n):
    city_data = api.geo_search(query = city, granularity = "city")
    
    for i, v in enumerate(city_data):
        print(i, v.full_name)
        
    num = int(input('Select the number:'))
    
    tweets = []
    for tweet in tweepy.Cursor(api.search, q = "place:%s" % city_data[num].id, lang = "es").items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def get_profile(user):
    profile = api.get_user(user)

    user = profile.name
    loc = profile.location
    id_me = profile.id
    created = profile.created_at.date()
    profile_img = profile.default_profile_image
    favs = profile.favourites_count
    followers = profile.followers_count
    friends = profile.friends_count
    url_prof_des = urls_user(profile)
    geo = profile.geo_enabled
    profile_img = profile.profile_image_url_https
    background_img = profile.profile_background_image_url_https
        
    profile = {'Id': id_me, 'Name': user, 'Location': loc, 'Created': created, 'Profile image': profile_img, 
               'Favs count': favs, 'Followers': followers, 'Following': friends, 'Geolocation': geo, 
               'Profile image': profile_img, 'Background image': background_img, 'URL profile description': url_prof_des}
    return profile

# Api authentication

First we read our credentials from the file **config_twitter.py** and we conect to twitter through the api.

In [12]:
import config_twitter

auth = OAuthHandler(config_twitter.consumer_key, config_twitter.consumer_secret)
auth.set_access_token(config_twitter.access_token, config_twitter.access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# Profile information

## Our profile

We can obtain our profile information.

In [14]:
my_profile()

{'Id': 1363194821490446339,
 'Name': 'Pablo',
 'Location': '',
 'Created': datetime.date(2021, 2, 20),
 'Profile image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'Favs count': 1,
 'Followers': 1,
 'Following': 1,
 'Geolocation': False,
 'Background image': None}

## Other profiles

Also we can get the information from any user profile using its username.

In [ ]:
get_profile('Tesla')

# Tweets

We can make different filters for tweets. For each tweet we obtain the date and time in which it is published, the unique id, the user who publishes it, the language, the number of retweets and favorites, the coordinates from where it is published, if it has been a response to any user and the hastags, urls, mentioned users and images or videos it contains.

## Last tweets

We get the last N tweets posted by users we are following.

In [ ]:
last_n_tweets(10)

## Last tweets user

We get the last N tweets posted by an user that we specify by its username.

In [ ]:
last_n_tweets_user('elonmusk', 10)

## Last tweets word

We get the last N tweets posted that contain a word.

In [ ]:
last_n_tweets_word('covid', 10)

## Last tweets location

We get the last N tweets posted from a location.

In [ ]:
last_n_tweets_city('Valencia', 10)